In [2]:
import os
import shutil

In [3]:
try:
    os.mkdir('zip')
except:
    pass
os.chdir('zip')
try:
    os.mkdir('../pdf_new')
except:
    pass
for folder in os.listdir(os.getcwd()):
    if folder.endswith('.pdf'):
        shutil.move(folder,os.getcwd()[:-3]+'pdf_new/'+folder)
    elif folder.endswith('ipynb'):
        continue
    else:            
        for file in os.listdir(os.getcwd()+'/'+folder):
            shutil.move(os.getcwd()+'/'+folder+'/'+file,os.getcwd()[:-3]+'pdf_new/'+file)
        os.rmdir(folder)
os.chdir('../') 
os.rmdir('zip')